In [1]:
! pip install transformers
! pip install torch torchvision torchaudio

  Using cached torchvision-0.19.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.0 kB)
  Using cached torchaudio-2.4.1-cp311-cp311-manylinux1_x86_64.whl.metadata (6.4 kB)
  Using cached torch-2.4.1-cp311-cp311-manylinux1_x86_64.whl.metadata (26 kB)
Using cached torchvision-0.19.1-cp311-cp311-manylinux1_x86_64.whl (7.0 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 797.1/797.1 MB 255.1 kB/s eta 0:00:0000:0100:57
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 629.2 kB/s eta 0:00:0000:0100:01
  Attempting uninstall: torch
    Found existing installation: torch 2.4.0
    Uninstalling torch-2.4.0:
      Successfully uninstalled torch-2.4.0


In [2]:
# MODEL = 'fine_tuned/NER/ACD_1epoch'
# MODEL = 'fine_tuned/NER/ACD_5epochs'
# MODEL = 'fine_tuned/NER/ACD_10epochs'

# MODEL = 'fine_tuned/NER/ACD'
# MODEL = 'ACD(overfited)'

# MODEL_PATH = '../../models/' + MODEL

MODEL_PATH = 'bert-base-cased'

In [3]:
import os, json
from transformers import AutoConfig

config = AutoConfig.from_pretrained(MODEL_PATH)

labels = []
labels = [value for k, value in config.id2label.items()]

print(config.id2label)

{0: 'LABEL_0', 1: 'LABEL_1'}


In [4]:
from transformers import BertForTokenClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, do_lower_case=False)

model = BertForTokenClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(labels),
    output_attentions = False,
    output_hidden_states = False
)

/opt/conda/lib/python3.11/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
# Separa o dataframe por PONTO-FINAL
def separar_frases(sentence):
    words = sentence.split(' ')
    return words

In [6]:
def tokenize(sentence):
    print(sentence)
    tokenized_sentence = []
    for word in sentence:
#         print('sentence ', word)
        # Tokenize the word and count # of subwords the word is broken into
#         print(word)
#         break
        tokenized_word = tokenizer.tokenize(str(word))
        print(tokenized_word)
        n_subwords = len(tokenized_word)
        
        # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

#         # Add the same label to the new list of labels `n_subwords` times
#         if label.startswith("B"):
#             labels.extend([label])
#             new_label = "I-" + label[2:]

#             labels.extend([new_label] * (n_subwords-1))
#         else:
#             labels.extend([label] * n_subwords)


    return tokenized_sentence


In [7]:
import torch
import numpy as np

test_sentence = "Neurological"


tokenized_sentence = tokenizer.encode(test_sentence)

# input_ids = torch.tensor([tokenized_sentence]).cuda()
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = model(input_ids)

label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)

tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
versum = ""

print(label_indices[0])
# print(len(tokens))

# for token, label_idx in zip(tokens, label_indices[0]):
entitying = False
change_of_label = False
current_label = ""
for idx, (token, label_idx) in enumerate(zip(tokens, label_indices[0])):
#     print(token)    
#     print(label_idx)
#     print(labels[label_idx])

#     print()
    if (token == "[CLS]" or token == "[SEP]"):
        continue
        
    if token.startswith("##"):
        sub_token = True
        versum = versum + token[2:]
#         new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
        if (labels[label_idx].startswith("B")):
            if entitying:
                versum += '}(' + entity +')'
                
            entitying = True
            versum = versum + ' {' + token
#             print(versum)
            entity = labels[label_idx][2:]
            
        
        if labels[label_idx].startswith("O") :
            if entitying:
#                 print(entity)
                versum = versum + '}(' + entity +') ' + token
            else:
                versum = versum + ' ' + token
            entitying = False

                
        if labels[label_idx].startswith("I") :
            if previous_label[2:] != labels[label_idx][2:]:
                versum += '}(' + entity +') {'
                entity = labels[label_idx][2:]
                change_of_label = True
#                 entitying = False
            else: change_of_label = False
                
#             print(labels[label_idx][2:])
#             if entitying:
            if change_of_label:
                versum += token
                entitying = True
            else:
    #       print(entity)
    #                 versum = versum + '}(' + entity +')'
                print('aqui')
                entitying = True
                versum += ' ' + token

#             else:
#                 versum = versum + ' ' + token
    previous_label = labels[label_idx]
        
#             new_labels.append(labels[label_idx])
#             new_tokens.append(token)
#     print(versum)

    
print(versum)


[0 1 1 1 0]
eurological


In [12]:
import torch
import numpy as np

# test_sentence = 'Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breathe and when I lay down on my bed. I suffer from arterial hypertension and smoke 20 cigarettes every day. My father had a “heart attack” at my age and I am very worried about it.'
# test_sentence = " Her eye is green. A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the eyes, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes. Alprazolan was administred."
# test_sentence = 'Recently, we found that therapy with selegiline and L-dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline. '
# test_sentence = 'The cells were examined in a Zeiss LSM 510 laser scanning microscope equipped with a Plan - Apochromate 63x/1.4 oil immersion objective.'
# test_sentence = 'Torsade de pointes ventricular tachycardia during  low dose intermittent dobutamine treatment in a patient with dilated cardiomyopathy and congestive heart failure.'
# test_sentence = 'A DNA molecule'
# test_sentence = '(a) Schematic drawing of the magnetic tweezers.'
# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. I have arterial hypertension but I don’t take my medication every day.'
# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. It has started 2 hours ago, but its intensity isn’t decreasing. It started suddenly and I am also feeling pain in my back and my neck. The pain is continuous, and it did not worse when I am breathing. I have arterial hypertension but I don’t take my medication every day, I am sorry! I don’t smoke, and I don’t know If I have other diseases. My father has also arterial hypertension.'
# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. It started suddenly and I am also feeling pain in my back and my neck. I have arterial hypertension but I don’t take my medication every day.'

test_sentence = 'Neurological'




# print(test_sentence)
tokenized_sentence = tokenizer.encode(test_sentence)
# print(tokenized_sentence)
# input_ids = torch.tensor([tokenized_sentence]).cuda()
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = model(input_ids)
# print(output)


label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
# print(label_indices)
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
print(tokens)
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
#     print(label_idx)
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
#         print('else')
#         print(new_labels)
#         print(new_tokens)
        new_labels.append(labels[label_idx])
        new_tokens.append(token)
for token, label in zip(new_tokens, new_labels):
    print("{}\t\t{}".format(token, label))


['[CLS]', 'N', '##eur', '##ological', '[SEP]']
[CLS]		LABEL_0
Neurological		LABEL_1
[SEP]		LABEL_0


In [9]:
test_sentence = '[CLS] '
# test_sentence += 'In two of the three deaths probably associated with ketoconazole treatment the drug had been continued after the onset of jaundice and other symptoms of hepatitis.'
test_sentence += 'Two patients with type II diabetes mellitus developed and acute hepatites-like syndrome soon after initiation of glyburide therapy.'
test_sentence += '[SEP]'
words = separar_frases(test_sentence)
# tokenize(words)
# print()

In [10]:
def tokenize_and_preserve_labels(sentence):
    tokenized_sentence = []
    labels = []
    for word in sentence:
#         print('sentence ', word)
        # Tokenize the word and count # of subwords the word is broken into
#         print(word)
#         break
        tokenized_word = tokenizer.tokenize(str(word))
        n_subwords = len(tokenized_word)

#         # Add the tokenized word to the final tokenized word list
        tokenized_sentence.extend(tokenized_word)

#         # Add the same label to the new list of labels `n_subwords` times
#         if label.startswith("B"):
#             labels.extend([label])
#             new_label = "I-" + label[2:]

#             labels.extend([new_label] * (n_subwords-1))
#         else:
#             labels.extend([label] * n_subwords)


    return tokenized_sentence